In [6]:
from src.metagpt import BioformatsReader
import javabridge
import bioformats
import ome_types

javabridge.start_vm(class_path=bioformats.JARS)

image_path = "/home/aaron/Documents/Projects/MetaGPT/in/images/testetst_Image8_edited_.ome.tif"
ome_xml = BioformatsReader.get_omexml_metadata(image_path)
ome_raw = BioformatsReader.get_raw_metadata(image_path)
ome_tree = BioformatsReader.raw_to_tree(ome_raw)
ome_dict = ome_types.to_dict(ome_xml)
print(ome_dict)

javabridge.kill_vm()

11:47:00.248 [Thread-0] DEBUG loci.common.NIOByteBufferProvider - Using mapped byte buffer? false
11:47:00.279 [Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader
java.lang.ClassNotFoundException: loci.formats.in.URLReader
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:331)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:264)
	at loci.formats.ClassList.parseLine(ClassList.java:196)
	at loci.formats.ClassList.parseFile(ClassList.java:258)
	at loci.formats.ClassList.<init>(ClassList.java:138)
11:47:00.285 [Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.SlideBook6Reader
java.lang.ClassNotFoundException: loci.formats.in.SlideBook6Reader
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lan

In [ ]:
from ome_types._autogenerated.ome_2016_06 import OME
from ome_types._autogenerated.ome_2016_06 import Pixels
from pydantic import BaseModel
from typing import Optional
from marvin.beta import Application

app = Application(
    name='OME Metadata Store',
    instructions=("Incoming metadata will be provided in raw format, that mean a list of key value pairs.!!!"
                  "Your task  will be to translate these key value pairs to the appropriate OME schema property(try to figure out which porperty is which by looking at the schema and the raw metadata in holistic manner). You will be "
                  "handed only a part of the ome schema to fill in, to reduce the scope."),
    state=OME(),
)

In [ ]:
print(app.thread)

In [ ]:
class ToDo(BaseModel):
    name: str
    done: bool = False

class ToDoState(BaseModel):
    todos: list[ToDo] = []


# --- create the application
todo_app = Application(
    name="ToDo App", instructions="A todo application", state=ToDoState()
)


In [ ]:
class MaybePixels(BaseModel):
    pixels: Optional[Pixels] = None

app = Application(
    name='OME Metadata Store',
    instructions=("Incoming metadata will be provided in raw format, that mean a list of key value pairs.!!!"
                  "Your task  will be to translate these key value pairs to the appropriate OME schema property(try to figure out which porperty is which by looking at the schema and the raw metadata in holistic manner). You will be "
                  "handed only a part of the ome schema to fill in, to reduce the scope.Since this is a hard problem I will need you to think step by step and use chain of thought to solve this problem. Here are some example on how to approach it:"
                  "Since the tool usage requires a python middle layer the usually used property names are written a bit differently, for example SizeX is written as size_x etc. The validation tool will make you aware of the correct property names."
                  "1. Look at the schema which elements are missing which one do you plan to add"
                  "2. Choose the easiest to implement element to start, Figure out if they have mandatory fields or dependencies. Try to keep the scope per change as small as possible so validation is easier."
                  "3. Look at the raw metadata, speicifcally try to find if any of the raw metadata would fit into the previously identified fields"
                  "4. Come to a conclusion wheter you can add the element or if there are mandatory fields missing"
                  "5. If you cant add the element start from step 1 again"
                  "6. If you can add the element, generate the tool call to add the element, make sure to add the element and all the mandatory fields to not get a validation error. Think about which operation is best fitting, are you addding a new node or modifying an existing one=?"
                  "7. Call the tool"
                  "8. If validation errors occur approach them systematically, is there a mandatory node missing? Is it possible to split the tool call into several steps? is there a dependency missing? is the value of the node correct? If you cant solve the problem start from step 1 again"
                  "9. Repeat the process until all elements are added and the metadata is valid. Remember to solve this problem step by step and use chain of thought to solve it. Good luck!"
                  ),
    state=OME()
)

In [ ]:
from ome_types import OME
from ome_types import Pixels

class MaybePixels(BaseModel):
    pixels: Optional[Pixels] = None

In [ ]:
MaybePixels().model_json_schema()

In [ ]:
await app.say_async("here is the raw metadata: {}".format(ome_raw))

In [ ]:
app.state.update_state_jsonpatches([{"op": "add",
                                        "path": "",
                                        "value": {
                                            "id": "Pixels:0",
                                            "SizeX": 512,
                                            "SizeY": 512,
                                            "SizeZ": 1,
                                            "SizeC": 1,
                                            "SizeT": 1,
                                            "DimensionOrder": "XYZCT",
                                            "PixelType": "uint8",
                                            "BigEndian": False}}])

In [ ]:
myome =OME()


In [ ]:
myome.images.append(Image(Pixels(PixelsType="uint8", SizeX=512, SizeY=512, SizeZ=1, SizeC=1, SizeT=1, DimensionOrder="XYZCT", BigEndian=False)))

In [ ]:
from ome_types._autogenerated.ome_2016_06 import Image
from ome_types._autogenerated.ome_2016_06 import Pixels
Pixels(id="Pixel1", dimension_order="XYZCT", type="uint8", size_x=512, size_y=512, size_z=512, size_c=1, size_t=1).model_dump_json()

In [ ]:
# Answer App
app_answer = Application(
    name='DataProviderGPT',
    instructions="You have a dictinoary of raw metadata as your state, your task will be to answer questions about the metadata.",
    state=ome_tree,
)




In [ ]:
app_answer.say("Does the metadata contain info about an image?")

In [ ]:
a = []
def test(a=0, b=0, c=0):
    return a + b + c

test(*a)

In [ ]:
# Define your Pydantic models
class Pixels(BaseModel):
    dimension_order: str
    type: str

class Image(BaseModel):
    pixels: Pixels
    name: str

class Experimenter(BaseModel):
    name: Optional[str] = None
    email: Optional[str] = None

class OME(BaseModel):
    images: List[Image]
    experimenters: List[Experimenter]

In [ ]:
from typing import List, Dict, Set, Type
from pydantic import BaseModel, Field
from marvin.beta import Application
from collections.abc import Iterator

promp1="""
Incoming metadata will be provided in raw format, that mean a list of key value pairs.!!!
Your task  will be to translate these key value pairs to the appropriate OME schema property(try to figure out which porperty is which by looking at the schema and the raw metadata in holistic manner). You will be
handed only a part of the ome schema to fill in, to reduce the scope.Since this is a hard problem I will need you to think step by step and use chain of thought to solve this problem. Here are some example on how to approach it:
Since the tool usage requires a python middle layer the usually used property names are written a bit differently, for example SizeX is written as size_x etc. The validation tool will make you aware of the correct property names.
1. Look at the schema which elements are missing which one do you plan to add
2. Figure out if they have mandatory fields or dependencies
3. Look at the raw metadata, speicifcally try to find if any of the raw metadata would fit into the previously identified fields
4. Come to a conclusion wheter you can add the element or if there are mandatory fields missing
5. If you cant add the element start from step 1 again
6. If you can add the element, generate the tool call to add the element, make sure to add the element and all the mandatory fields to not get a validation error. Think about which operation is best fitting, are you addding a new node or modifying an existing one=?
7. Call the tool
8. If validation errors occur approach them systematically, is there a mandatory node missing? is there a dependency missing? is the value of the node correct? If you cant solve the problem you can start from step 1 again
9. Repeat the process until all elements are added and the metadata is valid. Remember to solve this problem step by step and use chain of thought to solve it. Good luck!
I have split the ome model into several submodels, so if you feel like therer is not metadata for the data model you are working on, no worries just skip that model. Its very well possible some fields remain empty.
"""

# Define the tree node class
class TreeNode:
    def __init__(self, model: Type[BaseModel]):
        self.model = model
        self.object = None
        self.children = []

    def add_child(self, child: 'TreeNode'):
        self.children.append(child)

    def __repr__(self):
        return f"TreeNode(model={self.model.__name__}, children={self.children})"
    
    
    def required_fields(self, model: type[BaseModel], recursive: bool = False) -> Iterator[str]:
        """
        https://stackoverflow.com/questions/75146792/get-all-required-fields-of-a-nested-pydantic-model
        """
        for name, field in model.model_fields.items():
            if not field.is_required():
                continue
            t = field.annotation
            if recursive and isinstance(t, type) and issubclass(t, BaseModel):
                yield from self.required_fields(t, recursive=True)
            else:
                yield name
    
    def instantiate_model(self, child_objects) -> BaseModel:
        if any(self.required_fields(self.model)):
            print(any(self.required_fields(self.model)))
            # Create a MaybeModel class with a dynamic name
            maybe_model_name = f"Maybe{self.model.__name__}"
            #maybe_object_name = f"maybe{self.model.__name__}"

            annotations = {self.model.__name__: Optional[self.model]}
            MaybeModel = type(maybe_model_name,
                              (BaseModel,),
                              {'__annotations__': annotations,
                               self.model.__name__: Field(default=None,
                                                          description="The actual object to be filled with metadata.")})

            self.object = MaybeModel()  # we will need to fill in child objects later
        else:
            print(any(self.required_fields(self.model)))
            self.object = self.model(**child_objects)
            # print what kind of object this is
            print(self.object.model_dump_json())

    def predict_meta(self, raw_meta) -> BaseModel:
        # add the metdata from the child nodes first
        child_objects = {}
        for child in self.children:
            child_objects[child.model.__name__] = child.predict_meta(raw_meta)
        
        self.object = self.instantiate_model(child_objects)
        
        print(f"Predicting metadata for {self.model.__name__}, self.object={type(self.object)}, required={list(self.required_fields(self.model))}")
        return self.object
        app = Application(
            name='OME Metadata Store',
            instructions=(promp1),
            state=self.object,
        )
        app.say("here is the raw metadata: {}".format(raw_meta))
        
        return self.object

# Function to identify dependencies and collect all models
def collect_dependencies(model: Type[BaseModel], known_models: Dict[str, Type[BaseModel]], collected: Dict[str, Type[BaseModel]]):
    if model.__name__ in collected:
        return
    collected[model.__name__] = model
    for field in model.model_fields.values():
        field_type = field.annotation
        if hasattr(field_type, '__fields__'):  # If the field is a Pydantic model
            collect_dependencies(field_type, known_models, collected)
        elif hasattr(field_type, '__origin__') and field_type.__origin__ is list:
            item_type = field_type.__args__[0]
            if hasattr(item_type, '__fields__'):
                collect_dependencies(item_type, known_models, collected)

# Function to create the tree
def create_dependency_tree(model: Type[BaseModel], known_models: Dict[str, Type[BaseModel]], visited: Set[str]) -> TreeNode:
    node = TreeNode(model)
    visited.add(model.__name__)
    
    for field in model.model_fields.values():
        field_type = field.annotation
        if hasattr(field_type, '__fields__') and field_type.__name__ not in visited:  # If the field is a Pydantic model
            child_node = create_dependency_tree(field_type, known_models, visited)
            node.add_child(child_node)
        elif hasattr(field_type, '__origin__') and field_type.__origin__ is list:
            item_type = field_type.__args__[0]
            if hasattr(item_type, '__fields__') and item_type.__name__ not in visited:
                child_node = create_dependency_tree(item_type, known_models, visited)
                node.add_child(child_node)
    
    return node

# Function to build the tree starting from the root model
def build_tree(root_model: Type[BaseModel]) -> TreeNode:
    known_models = {model.__name__: model for model in globals().values() if isinstance(model, type) and issubclass(model, BaseModel)}
    collected_models = {}
    collect_dependencies(root_model, known_models, collected_models)
    
    return create_dependency_tree(root_model, known_models, set())

# Build the tree starting from the OME model
dependency_tree = build_tree(OME)

# Print the tree structure
def print_tree(node: TreeNode, indent: str = ""):
    print(f"{indent}{node.model.__name__}")
    for child in node.children:
        print_tree(child, indent + "  ")

print_tree(dependency_tree)


In [ ]:
dependency_tree.predict_meta(ome_raw)

In [ ]:
dependency_tree.children

In [ ]:
from ome_types import OME

dependency_tree = build_tree(OME)

dependency_tree.predict_meta(ome_raw)

In [ ]:
from ome_types import OME
from ome_types._autogenerated.ome_2016_06 import Image
from ome_types._autogenerated.ome_2016_06 import Pixels

myome = OME(images=[Image(pixels=Pixels(dimension_order='XYZCT', type='uint8', size_x=512, size_y=512, size_z=1, size_c=1, size_t=1))])

app = Application(
    name='OME Metadata Store',
    instructions=(promp1),
    state=myome,
)


In [ ]:
app.state.value.images.append(Image(pixels=Pixels(dimension_order='XYZCT', type='uint8', size_x=512, size_y=512, size_z=1, size_c=1, size_t=1)))

In [ ]:
pix = Pixels(dimension_order='XYZCT', type='uint8', size_x=512, size_y=512, size_z=1, size_c=1, size_t=1)

In [ ]:
Image.model_fields

In [ ]:
pix_dic = {type(pix).__name__.lower(): pix}
pix_dic
img = Image(**pix_dic)
ome = OME()


In [ ]:
obj = None
if obj:
    print("yes")

In [ ]:
def add_to_instance(instance, obj):
    mod_dic = {}
    # Get the type of the object
    obj_type = type(obj)
    
    # Define a set of ignored types
    ignored_types = {str, int, float, bool} # could be updated to != ome_types
    # Skip ignored types
    if obj_type in ignored_types:
        return
    # Iterate over the attributes and their types in the instance's class
    for attr_name, attr_type in instance.__annotations__.items():
        
        if attr_type == Optional[obj_type] or attr_type == obj_type:
            mod_dic[attr_name] = obj
            setattr(instance, attr_name, obj)
            return
        elif hasattr(attr_type, '__origin__') and attr_type.__origin__ == list and attr_type.__args__[0] == obj_type:
            getattr(instance, attr_name).append(obj)
            return
    

In [ ]:
my_img = Image
add_to_instance(my_img, pix)

In [ ]:
Image.__annotations__

In [ ]:
from src.metagpt.predictors import predictor_tree
from ome_types._autogenerated.ome_2016_06 import Image, Pixels

dependency_tree = predictor_tree.build_tree(Pixels)
state_predicted = dependency_tree.predict_meta(ome_raw)

In [ ]:
from datetime import datetime

datetime.now().isoformat()

In [ ]:

# Example usage:
output_path = 'output.json'
with save_cell_output(output_path):
    # Your code here
    print("This is an example output.qweqweq")
    # Any other code whose output you want to capture


In [ ]:
from src.metagpt.utils import save_and_stream_output
from time import sleep
with save_and_stream_output():
    # Your code here
    print("This is an example output.")
    # Any other code whose output you want to capture
    for i in range(5):
        sleep(1)
        print(f"Line {i + 1}")

In [ ]:
Experimenter.__name__

In [ ]:
from pydantic import BaseModel
from typing import Optional
from ome_types._autogenerated.ome_2016_06 import Image, Pixels, OME, Experimenter

class MaybeExperimenter(BaseModel):
    Experimenter: Optional[Experimenter] = None

In [ ]:
from ome_types._autogenerated.ome_2016_06 import Image, Pixels, OME, Experimenter
with save_and_stream_output():
    exp = Experimenter(id="Exp1")
    model = Experimenter
    object = MaybeExperimenter()
    object.Experimenter = exp
    object = getattr(object, model.__name__)
    print(type(object))

In [ ]:
f = [[]]

for i in f:
    print(i)

In [ ]:
from ome_types._autogenerated.ome_2016_06 import Image, Pixels, OME, Experimenter
from src.metagpt.predictors import predictor_tree
from src.metagpt.utils import save_and_stream_output

with save_and_stream_output():
    tree = predictor_tree.build_tree(OME)


In [ ]:
predictor_tree.print_tree(tree)

In [ ]:
# get the n attributes of an object:
# 
# 
for v in metadata.experimenters[0].__dict__.values():
    if v:
        print(v)

In [ ]:
len([v for v in metadata.experimenters[0].__dict__.values() if v])

In [ ]:
prompt = "qweqw"

In [ ]:
ome_xml

In [ ]:
ome_tree

In [ ]:
import marvin.utilities

In [ ]:
from ome_types._autogenerated.ome_2016_06 import StructuredAnnotations
from marvin.beta import Application

prompt = """
You are a metadata curation AI. The OME model deoesnt have corresponding properties for all metadata in its datamodel.
This leads to lost metadata if a file was originally stored in a proprietary format.
Your task is to save this lost  data and structure it as structured annotation in the OME XML format.
You will be provided with the raw metadata and the OMEXML file.
You will need to find the differences between the two and fill in the missing metadata in the OME XML structured annotations.
Since this is a hard task solve it step by step and use chain of thought.
In particular, you should:
1. Start by comparing the raw metadata with the provided ome xml. Find metadata which is not present in the ome xml.
2. Think of a way to structure this metadata in the structured annotations, You can change the names of nodes in the structured annotations to fit the metadata. Try to match the ome xml style.
3. Add the missing metadata to the structured annotations.
There will be a validation tool validating your input, if there are mistakes you will be informed.
Carefully fix those mistakes, make sure to adhere to the required style.
"""
sa = StructuredAnnotations()
app = Application(
    name='OME Metadata Store',
    instructions=(prompt),
    state=sa,
)

app.say("here is the raw metadata: {}".format(ome_tree) + "\n here is the ome xml metadata: {}".format(ome_xml))

In [7]:
from openai import OpenAI
client = OpenAI()

prompt = """
You are a metadata curation AI. The OME model deoesnt have corresponding properties for all metadata in its datamodel.
This leads to lost metadata if a file was originally stored in a proprietary format.
Your task is to save this lost  data and structure it as structured annotation in the OME XML format.
You will be provided with the raw metadata and the OMEXML file.
You will need to find the differences between the two and fill in the missing metadata in the OME XML structured annotations.
Since this is a hard task solve it step by step and use chain of thought.
In particular, you should:
1. Start by comparing the raw metadata with the provided ome xml. Find metadata which is not present in the ome xml.
2. Think of a way to structure this metadata in the structured annotations, You can change the names of nodes in the structured annotations to fit the metadata. Try to match the ome xml style.
3. Add the missing metadata to the structured annotations.
There will be a validation tool validating your input, if there are mistakes you will be informed.
Carefully fix those mistakes, make sure to adhere to the required style.
"""

assistant = client.beta.assistants.create(
  name="Metadata Curation Assistant",
  instructions=prompt,
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="here is the raw metadata: {}".format(ome_tree) + "\n here is the ome xml metadata: {}".format(ome_xml)
)

In [8]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="follow the instructions closely and fill in the missing metadata in the structured annotations",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Let's integrate the provided raw metadata into a more structured format while taking into account the supplied OME XML metadata.

### Structured Metadata
```json
{
  "Image": {
    "ID": "Image:0",
    "Name": "testetst_Image8_edited_.ome.tif",
    "Description": "",
    "Pixels": {
      "BigEndian": "false",
      "DimensionOrder": "XYCZT",
      "ID": "Pixels:0",
      "Interleaved": "false",
      "PhysicalSizeX": "0.09922878199885109",
      "PhysicalSizeXUnit": "µm",
      "PhysicalSizeY": "0.09922878199885109",
      "PhysicalSizeYUnit": "µm",
      "SignificantBits": "8",
      "SizeC": "3",
      "SizeT": "30",
      "SizeX": "680",
      "SizeY": "280",
      "SizeZ": "1",
      "Type": "uint8",
      "Channels": [
        {"ID": "Channel:0:0", "SamplesPerPixel": "1", "LightPath": {}},
        {"ID": "Channel:0:1", "SamplesPerPixel": "1", "LightPath": {}},
        {"ID": "Channel:0:2", "SamplesPerPixel": "1", "LightPath": {}}
      ],
      "MetadataOnly": true
 

In [10]:
ome_xml

'<?xml version="1.0" encoding="UTF-8"?><OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd"><Image ID="Image:0" Name="testetst_Image8_edited_.ome.tif"><Description/><Pixels BigEndian="false" DimensionOrder="XYCZT" ID="Pixels:0" Interleaved="false" PhysicalSizeX="0.09922878199885109" PhysicalSizeXUnit="µm" PhysicalSizeY="0.09922878199885109" PhysicalSizeYUnit="µm" SignificantBits="8" SizeC="3" SizeT="30" SizeX="680" SizeY="280" SizeZ="1" Type="uint8"><Channel ID="Channel:0:0" SamplesPerPixel="1"><LightPath/></Channel><Channel ID="Channel:0:1" SamplesPerPixel="1"><LightPath/></Channel><Channel ID="Channel:0:2" SamplesPerPixel="1"><LightPath/></Channel><MetadataOnly/></Pixels></Image><StructuredAnnotations><XMLAnnotation ID="Annotation:0" Namespace="openmicroscopy.org/OriginalMetadata"><Value><Origi

In [9]:
ome_tree

{'XResolution': '100777.212',
 'PhotometricInterpretation': 'BlackIsZero',
 'PlanarConfiguration': 'Chunky',
 'MetaDataPhotometricInterpretation': 'Monochrome',
 'ImageLength': '280',
 'BitsPerSample': '8',
 'ResolutionUnit': 'Centimeter',
 'hyperstack': 'true',
 'Compression': 'Uncompressed',
 'SamplesPerPixel': '1',
 'NumberOfChannels': '1',
 'Software': 'OME Bio-Formats 6.7.0',
 'MetaMorph': 'no',
 'SampleFormat': 'unsigned integer',
 'ImageWidth': '680',
 'ImageJ': '',
 'YResolution': '100777.212'}

In [ ]:
sa.model_dump_json()

In [ ]:
import anthropic

anthropic.Anthropic().messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, world"}
    ]
)

print(message.content)


In [18]:
StructuredAnnotations.model_json_schema()

{'$defs': {'AnnotationRef': {'description': 'The AnnotationRef element is a reference to an element derived from the\nCommonAnnotation element.',
   'properties': {'id': {'default': '__auto_sequence__',
     'name': 'ID',
     'pattern': '(urn:lsid:([\\w\\-\\.]+\\.[\\w\\-\\.]+)+:Annotation:\\S+)|(Annotation:\\S+)',
     'required': True,
     'title': 'Id',
     'type': 'Attribute'}},
   'title': 'AnnotationRef',
   'type': 'object'},
  'BinData': {'description': 'The contents of this element are base64-encoded.\n\nThese are not CDATA sections, just a base64 stream.\n\nAttributes\n----------\nvalue : bytes\n    (The BinData value).\ncompression : BinData_Compression\n    Specifies the compression scheme used to encode the data.\nbig_endian : bool\n    This is true if the binary data was written in BigEndian order. This is\n    dependent on the system architecture of the machine that wrote the pixels.\n    True for essentially all modern CPUs other than Intel and Alpha. All Binary\n    

In [13]:
from src.metagpt.utils import openai_schema
from ome_types._autogenerated.ome_2016_06 import StructuredAnnotations
import instructor
from openai import OpenAI

client = instructor.patch(OpenAI(), mode=instructor.Mode.FUNCTIONS)


#my_tool_call = openai_schema(StructuredAnnotations)

client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user",
            "content": prompt + f"Here is the raw metadata: {ome_tree} and here is the ome xml metadata: {ome_xml}"},],
    response_model=
).model_dump()

/tmp/ipykernel_4340/2734795817.py:6: DeprecationWarning: FUNCTIONS is deprecated and will be removed in future versions
  client = instructor.patch(OpenAI(), mode=instructor.Mode.FUNCTIONS)


TypeError: unhashable type: 'StructuredAnnotations'

In [25]:
StructuredAnnotations().model_json_schema()

{'$defs': {'AnnotationRef': {'description': 'The AnnotationRef element is a reference to an element derived from the\nCommonAnnotation element.',
   'properties': {'id': {'default': '__auto_sequence__',
     'name': 'ID',
     'pattern': '(urn:lsid:([\\w\\-\\.]+\\.[\\w\\-\\.]+)+:Annotation:\\S+)|(Annotation:\\S+)',
     'required': True,
     'title': 'Id',
     'type': 'Attribute'}},
   'title': 'AnnotationRef',
   'type': 'object'},
  'BinData': {'description': 'The contents of this element are base64-encoded.\n\nThese are not CDATA sections, just a base64 stream.\n\nAttributes\n----------\nvalue : bytes\n    (The BinData value).\ncompression : BinData_Compression\n    Specifies the compression scheme used to encode the data.\nbig_endian : bool\n    This is true if the binary data was written in BigEndian order. This is\n    dependent on the system architecture of the machine that wrote the pixels.\n    True for essentially all modern CPUs other than Intel and Alpha. All Binary\n    

In [40]:
from pydantic import BaseModel, Field
from typing import Optional

class XML_Annotations(BaseModel):
    xml: Optional[dict] = Field(default_factory=dict, description="The structured annotation in XML format.")

In [38]:
from ome_types._autogenerated.ome_2016_06 import XMLAnnotation

XMLAnnotation(value=)

XMLAnnotation(
   id='Annotation:3',
   value={'any_elements': ['qweqweqwe']},
)

In [35]:
StructuredAnnotations(xml_annotations=[{}])

ValidationError: 1 validation error for StructuredAnnotations
xml_annotations.0.value
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [43]:
import marvin
from ome_types._autogenerated.ome_2016_06 import Pixels
from ome_types._autogenerated.ome_2016_06 import OME

sa = StructuredAnnotations()
msg = "Here is the raw metadata: {} and here is the ome xml metadata: {}".format(ome_tree, ome_xml)
test = marvin.extract(msg,
               target=XML_Annotations,)

In [63]:
XMLAnnotation.model_json_schema()

{'$defs': {'AnnotationRef': {'description': 'The AnnotationRef element is a reference to an element derived from the\nCommonAnnotation element.',
   'properties': {'id': {'default': '__auto_sequence__',
     'name': 'ID',
     'pattern': '(urn:lsid:([\\w\\-\\.]+\\.[\\w\\-\\.]+)+:Annotation:\\S+)|(Annotation:\\S+)',
     'required': True,
     'title': 'Id',
     'type': 'Attribute'}},
   'title': 'AnnotationRef',
   'type': 'object'},
  'Value': {'properties': {'any_elements': {'items': {},
     'namespace': '##any',
     'title': 'Any Elements',
     'type': 'Wildcard'}},
   'title': 'Value',
   'type': 'object'}},
 'description': 'An general xml annotation.\n\nThe contents of this is not processed as OME XML but should still be\nwell-formed XML.',
 'properties': {'description': {'anyOf': [{'type': 'string'},
    {'type': 'null'}],
   'default': None,
   'name': 'Description',
   'namespace': 'http://www.openmicroscopy.org/Schemas/OME/2016-06',
   'title': 'Description',
   'type': 'E

In [64]:
myxml = XMLAnnotation(value=f"{test[0].xml}")

In [71]:
sa = StructuredAnnotations(xml_annotations=[myxml])

In [72]:
type(sa)

ome_types._autogenerated.ome_2016_06.structured_annotations.StructuredAnnotations

In [73]:
from ome_types._autogenerated.ome_2016_06 import OME

OME(structured_annotations=sa)

OME(structured_annotations={'xml_annotations': [{'id': 'Annotation:13', 'value': {'any_elements': ["{'XResolution': '100777.212', 'PhotometricInterpretation': 'BlackIsZero', 'PlanarConfiguration': 'Chunky', 'MetaDataPhotometricInterpretation': 'Monochrome', 'ImageLength': 280, 'BitsPerSample': 8, 'ResolutionUnit': 'Centimeter', 'hyperstack': 'true', 'Compression': 'Uncompressed', 'SamplesPerPixel': 1, 'NumberOfChannels': 1, 'Software': 'OME Bio-Formats 6.7.0', 'MetaMorph': 'no', 'SampleFormat': 'unsigned integer', 'ImageWidth': 680, 'ImageJ': '', 'YResolution': '100777.212'}"]}, 'kind': 'xmlannotation'}]})

In [74]:
ome_types.to_xml(OME(structured_annotations=sa))

'<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">\n  <StructuredAnnotations>\n    <XMLAnnotation ID="Annotation:13">\n      <Value>{\'XResolution\': \'100777.212\', \'PhotometricInterpretation\': \'BlackIsZero\', \'PlanarConfiguration\': \'Chunky\', \'MetaDataPhotometricInterpretation\': \'Monochrome\', \'ImageLength\': 280, \'BitsPerSample\': 8, \'ResolutionUnit\': \'Centimeter\', \'hyperstack\': \'true\', \'Compression\': \'Uncompressed\', \'SamplesPerPixel\': 1, \'NumberOfChannels\': 1, \'Software\': \'OME Bio-Formats 6.7.0\', \'MetaMorph\': \'no\', \'SampleFormat\': \'unsigned integer\', \'ImageWidth\': 680, \'ImageJ\': \'\', \'YResolution\': \'100777.212\'}</Value>\n    </XMLAnnotation>\n  </StructuredAnnotations>\n</OME>\n'

In [79]:
type(ome_tree)

dict